In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
from sklearn.tree import DecisionTreeRegressor

In [3]:
#loading different data
train_datapath = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\train.csv"
train = pd.read_csv(train_datapath,)

magnetic_shielding_tensors_datapath = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\magnetic_shielding_tensors.csv"
mst = pd.read_csv(magnetic_shielding_tensors_datapath)

dipolemoment_datapath = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\dipole_moments.csv"
dm = pd.read_csv(dipolemoment_datapath)

test_path = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\test.csv"
test = pd.read_csv(test_path, index_col = 'type')

mulliken_charge_path = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\mulliken_charges.csv"
mc = pd.read_csv(mulliken_charge_path)

potentialenergy_path = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\potential_energy.csv"
pe = pd.read_csv(potentialenergy_path)

scalar_coupling_contributions_path = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\scalar_coupling_contributions.csv"
scc = pd.read_csv(scalar_coupling_contributions_path)

structures_path = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\structures.csv"
structures = pd.read_csv(structures_path)

In [4]:
#summary of the data(copied from Kernel)
print(f'There are {train.shape[0]} rows in train data.')
print(f'There are {test.shape[0]} rows in test data.')

print(f"There are {train['molecule_name'].nunique()} distinct molecules in train data.")
print(f"There are {test['molecule_name'].nunique()} distinct molecules in test data.")
print(f"There are {structures['atom'].nunique()} unique atoms.")
print(f"There are {train['type'].nunique()} unique types.")

There are 4658147 rows in train data.
There are 2505542 rows in test data.
There are 85003 distinct molecules in train data.
There are 45772 distinct molecules in test data.
There are 5 unique atoms.
There are 8 unique types.


In [5]:
train.head(30)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.80760
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.25700
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.25480
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.25430
4,4,dsgdb9nsd_000001,2,0,1JHC,84.80740
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.25410
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.25480
7,7,dsgdb9nsd_000001,3,0,1JHC,84.80930
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.25430
9,9,dsgdb9nsd_000001,4,0,1JHC,84.80950


In [6]:
test.head(15)

,id,molecule_name,atom_index_0,atom_index_1
type,,,,
2JHC,4658147,dsgdb9nsd_000004,2,0
1JHC,4658148,dsgdb9nsd_000004,2,1
3JHH,4658149,dsgdb9nsd_000004,2,3
1JHC,4658150,dsgdb9nsd_000004,3,0
2JHC,4658151,dsgdb9nsd_000004,3,1
1JHC,4658152,dsgdb9nsd_000015,3,0
3JHC,4658153,dsgdb9nsd_000015,3,2
2JHH,4658154,dsgdb9nsd_000015,3,4
2JHH,4658155,dsgdb9nsd_000015,3,5


In [7]:
mst.head()

,molecule_name,atom_index,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ
0,dsgdb9nsd_000001,0,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170
1,dsgdb9nsd_000001,1,31.3410,-1.2317,4.0544,-1.2317,28.9546,-1.7173,4.0546,-1.7173,34.0861
2,dsgdb9nsd_000001,2,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967
3,dsgdb9nsd_000001,3,31.5172,4.1086,1.2723,4.1088,33.9068,1.6950,1.2724,1.6951,28.9579
4,dsgdb9nsd_000001,4,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259,-1.1795,1.6260,28.9013


In [8]:
pe.head()#potential energy

,molecule_name,potential_energy
0,dsgdb9nsd_000001,-40.52368
1,dsgdb9nsd_000002,-56.56025
2,dsgdb9nsd_000003,-76.42608
3,dsgdb9nsd_000005,-93.42849
4,dsgdb9nsd_000007,-79.83869


In [9]:
structures.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [10]:
#merge the data
scc_train = train.merge(scc)
scc_train.to_csv("scc_train.csv", index = 'False')

In [11]:
scc_train.head(15)
#fc, sd, pso, dso mean different coupling mechanism

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,fc,sd,pso,dso
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,83.0224,0.254579,1.25862,0.272010
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,-11.0347,0.352978,2.85839,-3.433600
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,-11.0325,0.352944,2.85852,-3.433870
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,-11.0319,0.352934,2.85855,-3.433930
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,83.0222,0.254585,1.25861,0.272013
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.2541,-11.0317,0.352932,2.85856,-3.433950
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.2548,-11.0324,0.352943,2.85853,-3.433870
7,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,83.0241,0.254634,1.25856,0.272012
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.2543,-11.0319,0.352943,2.85856,-3.433930
9,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,83.0243,0.254628,1.25856,0.272012


In [12]:
#set up the X and y
y = scc_train.scalar_coupling_constant
data_features = ['atom_index_0','atom_index_1','type','fc','sd','pso','dso']
X = scc_train[data_features]
X.head()


,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,1,0,1JHC,83.0224,0.254579,1.25862,0.272010
1,1,2,2JHH,-11.0347,0.352978,2.85839,-3.433600
2,1,3,2JHH,-11.0325,0.352944,2.85852,-3.433870
3,1,4,2JHH,-11.0319,0.352934,2.85855,-3.433930
4,2,0,1JHC,83.0222,0.254585,1.25861,0.272013


In [13]:
#import model
from sklearn.ensemble import RandomForestRegressor
forest_model = RandomForestRegressor(random_state = 1)
#train test split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
scc_train_X, val_X, scc_train_y, val_y = train_test_split(X, y, random_state = 1)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [17]:
#deal with catagorical data
from sklearn.preprocessing import LabelEncoder

label_scc_train_X = scc_train_X.copy()
label_val_X = val_X.copy()#make copy to avoid chaging original data

encoder = LabelEncoder()
label_scc_train_X['type'] = encoder.fit_transform(scc_train_X['type'])
label_val_X['type'] = encoder.transform(val_X['type'])#apply label encder to each column with categprical data


In [23]:
#get mae
def mae(scc_train_X, val_X, scc_train_y, val_y):
    model = RandomForestRegressor(random_state = 1)
    model.fit(scc_train_X, scc_train_y)
    preds = model.predict(val_X)
    return mean_absolute_error(val_y, preds)

In [24]:
print(mae(label_scc_train_X, label_val_X, scc_train_y, val_y))

0.013660164932817619
